# Mozart Annotation

This notebook can be used to annotate schemata the Mozart piano sonatas.

The first part is some setup, which needs to run once but you can ignore.

In [ ]:
pwd()

In [ ]:
import Pkg; Pkg.activate("..")
using DigitalMusicology
using Interact
include("../src/polygrams.jl");

In [ ]:
corpusdir = joinpath("..", "data", "corpora", "dcml_corpora", "annotations", "Mozart-Sonatas")
DigitalMusicology.usedir(joinpath(corpusdir, "Schema Annotations", "musicxml"))
lexicon = Polygrams.loadlexicon(joinpath("..", "data", "lexicon_spelled_flat.json"));
annotdir = joinpath(corpusdir, "Schema Annotations", "annotations");
DigitalMusicology.allpieces()

# Start Here!!!

Annotation consists of three steps:
1. specification of piece and schema
2. annotation (matcher-aided + manual)
3. saving the annotations

## 1. Specify Piece and Schema

In order to annotate a piece wrt. a schema variant, adapt the following values, then run all cells up to **Annotate**.

In [ ]:
# adapt this:
piece = "K281-3"; #filename in data/corpora/mozart-piano-sonatas/midi-norep without extension
schema = "fonte.2";   # name of the schema as in data/lexicon_flat.json

Leave this code unchanged (but run the cells):

In [ ]:
notes = getpiece(piece, :notes, :musicxml, type=:notes, keepids=true);
xml = getpiece(piece, :xml, :musicxml, keepids=true)
timesigs = getpiece(piece, :timesigs, :musicxml)[1]
timesig = content(timesigs[1]);
barlen = duration(timesig)
beatfactor = denominator(timesig)
pattern = lexicon[schema];
featurefs = [
    poly -> beatfactor * Polygrams.totalduration(poly), # duration
    Polygrams.voicedist,                                # voice dist
    poly -> beatfactor * Polygrams.instageskip(poly)    # stage skip
    #, poly -> Polygrams.polymweight(poly, timesigs) # metric weight
]
featureweights = [1, -1, -2]; # maaaaaagic

In [ ]:
polyitr = Polygrams.schemamatches(notes, [pattern], barlen, barlen);
polys = collect(polyitr);
sorted = Polygrams.sortbyheuristics(polys, featurefs, featureweights);
scores = Dict(poly => Polygrams.rate(poly, featurefs, featureweights) for poly in polys);

In [ ]:
length(sorted)

## 2. Annotate

Run the following two cells to (re-) create an assisted and a manual annotation view.
Then annotate the schema instances by selecting and adapting the found instances in the assited view
and adding new instances in the manual view, if necessary.

In [ ]:
matcherwdg = Polygrams.matchinteractivewdg(notes, sorted, xml)

In [ ]:
markwdg = Polygrams.markschemaswdg(notes, xml, [pattern], timesigs=timesigs)

## 3. Save

Run the next two cells to save the piece. Make sure to not overwrite an old annotation by running this notebook!

In [ ]:
save = Interact.confirm("Are you sure you want to write (and potentially overwrite) the $(schema) annotations for $(piece)?") do yes
    if (yes)
        annots = collect(markwdg[])
        union!(annots, matcherwdg[])
        Polygrams.saveannots(piece, schema, annots, annotdir)
        println("saved!")
    end
end

In [ ]:
save()

# Repeat

If you want to annotate another piece, po back to **Start Here**.